In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime

In [58]:
# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Make sure chromedriver is in your PATH
driver.get("http://en.stockbiz.vn/IndicesStats.aspx#")

# Wait for the page to load
driver.implicitly_wait(10)

# Wait for the calendar widget to be fully initialized
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_dtStartDate_picker"))
)

<selenium.webdriver.remote.webelement.WebElement (session="d27a16704c37129cb1ef782441d6b9e7", element="f.F71EC71ADECEC04F50198EFDB1DCD609.d.669BA0645A3715B616FBEECC8E1EF7E4.e.481")>

In [59]:
# Locate the hidden start date input field
start_date_hidden_input = driver.find_element(By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_dtStartDate_picker_selecteddates")

# Locate the visible start date input field
start_date_visible_input = driver.find_element(By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_dtStartDate_picker_picker")

# Set the start date value using JavaScript (both hidden and visible fields)
start_date = "28/07/2000"  # Format: DD/MM/YYYY (adjust based on the website's expected format)
driver.execute_script(f"arguments[0].value = '{start_date}';", start_date_hidden_input)
driver.execute_script(f"arguments[0].value = '{start_date}';", start_date_visible_input)

# Simulate the ComponentArt Calendar widget's internal logic
# This JavaScript code sets the selected date in the calendar widget
calendar_widget_script = """
var calendar = window.ctl00_webPartManager_wp267165551_wp1192412521_dtStartDate_picker;
if (calendar && calendar.setSelectedDate) {
    var selectedDate = new Date(2000, 6, 28);  // Year, Month (0-based), Day
    calendar.setSelectedDate(selectedDate);
    if (calendar.render) {
        calendar.render();
    } else {
        console.error("render method not found on calendar widget.");
    }
} else {
    console.error("Calendar widget or setSelectedDate method not found.");
}
"""

# Execute the script to update the calendar widget
driver.execute_script(calendar_widget_script)

# Trigger events to ensure the website recognizes the change
driver.execute_script("arguments[0].dispatchEvent(new Event('input'));", start_date_hidden_input)
driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", start_date_hidden_input)
driver.execute_script("arguments[0].dispatchEvent(new Event('input'));", start_date_visible_input)
driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", start_date_visible_input)

# Locate the hidden end date input field
end_date_hidden_input = driver.find_element(By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_dtEndDate_picker_selecteddates")

# Locate the visible end date input field
end_date_visible_input = driver.find_element(By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_dtEndDate_picker_picker")

# Set the end date value using JavaScript (both hidden and visible fields)
end_date = datetime.today().strftime("%d/%m/%Y")  # Format: DD/MM/YYYY (adjust based on the website's expected format)
driver.execute_script(f"arguments[0].value = '{end_date}';", end_date_hidden_input)
driver.execute_script(f"arguments[0].value = '{end_date}';", end_date_visible_input)

# Simulate the ComponentArt Calendar widget's internal logic for the end date
calendar_widget_script_end_date = """
var calendar = window.ctl00_webPartManager_wp267165551_wp1192412521_dtEndDate_picker;
if (calendar && calendar.setSelectedDate) {
    var selectedDate = new Date();  // Today's date
    calendar.setSelectedDate(selectedDate);
    if (calendar.render) {
        calendar.render();
    } else {
        console.error("render method not found on calendar widget.");
    }
} else {
    console.error("Calendar widget or setSelectedDate method not found.");
}
"""

# Execute the script to update the calendar widget for the end date
driver.execute_script(calendar_widget_script_end_date)

# Trigger events to ensure the website recognizes the change
driver.execute_script("arguments[0].dispatchEvent(new Event('input'));", end_date_hidden_input)
driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", end_date_hidden_input)
driver.execute_script("arguments[0].dispatchEvent(new Event('input'));", end_date_visible_input)
driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", end_date_visible_input)

# Debugging: Print the updated values
start_date_hidden_value = driver.execute_script("return arguments[0].value;", start_date_hidden_input)
start_date_visible_value = driver.execute_script("return arguments[0].value;", start_date_visible_input)
end_date_hidden_value = driver.execute_script("return arguments[0].value;", end_date_hidden_input)
end_date_visible_value = driver.execute_script("return arguments[0].value;", end_date_visible_input)
print("Start Date (Hidden):", start_date_hidden_value)
print("Start Date (Visible):", start_date_visible_value)
print("End Date (Hidden):", end_date_hidden_value)
print("End Date (Visible):", end_date_visible_value)

# Locate and click the "View" button to refresh the data
view_button = driver.find_element(By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_btnView")
driver.execute_script("arguments[0].click();", view_button)  # Use JavaScript to click the button

# Wait for the data to load
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_callbackData"))
    )
    print("Data loaded successfully.")
except Exception as e:
    print("Data did not load:", e)

Start Date (Hidden): 2000.7.28
Start Date (Visible):  7/28/2000
End Date (Hidden): 2025.4.29
End Date (Visible):  4/29/2025
Data loaded successfully.


In [60]:
# Function to scrape data from the current page
def scrape_current_page(is_first_page):
    # Locate the table or data container
    table = driver.find_element(By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_callbackData")

    # Extract table rows
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Extract data into a list
    data = []
    for i, row in enumerate(rows):
        # Skip the footer row (last row)
        if i == len(rows) - 1:
            continue

        # Skip the title row (first row) for pages 2 and beyond
        if not is_first_page and i == 0:
            continue

        # Extract columns
        cols = row.find_elements(By.TAG_NAME, "td")
        cols = [col.text.strip() for col in cols[:-1]]
        data.append(cols)

    return data

# Function to get the total number of pages from the table footer
def get_total_pages():
    # Locate the table footer
    table_footer = driver.find_element(By.CLASS_NAME, "TableFooter")

    # Extract the text (e.g., "Page 1 of 299 Next>")
    footer_text = table_footer.text

    # Extract the total number of pages
    # Example: "Page 1 of 299 Next>" -> Extract "299"
    total_pages_text = footer_text.split("of")[1].strip()  # "299 Next>"
    total_pages = int(total_pages_text.split()[0])  # Extract "299" and convert to int
    return total_pages

def remove_outliers(df):
    # Extract percentage from Change column and convert to decimal
    df['Change'] = df['Change'].str.extract(r"\(([-+]?[\d.]+)%\)").astype(float) / 100

    # keep only rows where |Change| ≤ 0.03
    df = df[df['Change'].abs() <= 0.03]
    return df

In [61]:
# Initialize a list to store all data
all_data = []

# Scrape data from the first page (including the title row)
first_page_data = scrape_current_page(is_first_page=True)
title_row = first_page_data[0]  # Extract the title row
all_data.extend(first_page_data[1:])  # Append the rest of the data (excluding the title row)

# Get the total number of pages from the table footer
total_pages = get_total_pages()
print(f"Total pages to scrape: {total_pages}")

# Initialize the offset for pagination
offset = 20

# Loop through all pages
for page in range(2, total_pages + 1):  # Start from page 2 (since we already scraped page 1)
    try:
        # Locate the "Next" button
        next_button = driver.find_element(By.XPATH, "//a[contains(@onclick, 'RefreshData')]")

        # Update the "Next" button's onclick attribute to the correct offset
        driver.execute_script(f"arguments[0].setAttribute('onclick', 'RefreshData({offset})');", next_button)

        # Click the "Next" button using JavaScript
        driver.execute_script("arguments[0].click();", next_button)

        # Wait for the data to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ctl00_webPartManager_wp267165551_wp1192412521_callbackData"))
        )

        # Scrape data from the current page (excluding the title row)
        all_data.extend(scrape_current_page(is_first_page=False))
        print(f"Scraped data from page {page}.")

        # Increment the offset by 20 for the next page
        offset += 20

    except Exception as e:
        # If the "Next" button is not found or an error occurs, stop the loop
        print("No more pages or an error occurred:", e)
        break

# Convert the data into a pandas DataFrame
df = pd.DataFrame(all_data, columns=title_row)  # Use the title row as column headers

# # Add an index column
# df.reset_index(inplace=True)  # Add a sequential index column
# df.rename(columns={"index": "Index"}, inplace=True)  # Rename the index column to "Index"

df = remove_outliers(df)

# Save the data to a CSV file
df.to_csv("../vn_index/hose_historical_data.csv", index=False)  # Do not use any column as the index
print("Data saved to hose_historical_data.csv")

# Close the browser
driver.quit()

Total pages to scrape: 301
Scraped data from page 2.
Scraped data from page 3.
Scraped data from page 4.
Scraped data from page 5.
Scraped data from page 6.
Scraped data from page 7.
Scraped data from page 8.
Scraped data from page 9.
Scraped data from page 10.
Scraped data from page 11.
Scraped data from page 12.
Scraped data from page 13.
Scraped data from page 14.
Scraped data from page 15.
Scraped data from page 16.
Scraped data from page 17.
Scraped data from page 18.
Scraped data from page 19.
Scraped data from page 20.
Scraped data from page 21.
Scraped data from page 22.
Scraped data from page 23.
Scraped data from page 24.
Scraped data from page 25.
Scraped data from page 26.
Scraped data from page 27.
Scraped data from page 28.
Scraped data from page 29.
Scraped data from page 30.
Scraped data from page 31.
Scraped data from page 32.
Scraped data from page 33.
Scraped data from page 34.
Scraped data from page 35.
Scraped data from page 36.
Scraped data from page 37.
Scraped d

In [62]:
# # Read existing CSV
# df = pd.read_csv("../vn_index/hose_historical_data.csv")

# # Scrape the first page
# first_page_data = scrape_current_page(is_first_page=True)

# # Extract header and data
# title_row = first_page_data[0]
# first_page_df = pd.DataFrame(first_page_data[2:], columns=title_row)
# first_page_df = remove_outliers(first_page_df)

# # Combine the new and existing data
# combined_df = pd.concat([first_page_df, df], ignore_index=True)

# # Drop duplicates based on 'Date' column only — keep the first occurrence
# combined_df.drop_duplicates(subset=['Date'], keep='first', inplace=True)

# # Drop old 'Index' column if it's already there
# if 'Index' in combined_df.columns:
#     combined_df.drop(columns=['Index'], inplace=True)

# # Reset index and insert 'Index' column as a normal column
# combined_df.reset_index(drop=True, inplace=True)
# # combined_df.insert(0, 'Index', combined_df.index)

# # Save to CSV
# combined_df.to_csv("../vn_index/hose_historical_data.csv", index=False)
# print("Data saved to hose_historical_data.csv")

# # Quit driver
# driver.quit()

In [63]:
# first_page_df

In [64]:
# combined_df[:60]